In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 60.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 85.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 27.2 MB/s eta 0:00:00


In [ ]:
! pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=131f5439039912f64277da85d20b42badb12816585da8ad8a0c183f38282bd7e
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig, AutoModelForSeq2SeqLM

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("summarization", model="facebook/bart-large-cnn")

In [ ]:
import pandas as pd
data = pd.read_csv('reference_summary_original_text.csv')

In [ ]:
gold_summary = data['Enforcement Summary'].to_list()
original_text = data['text_extracted'].to_list()

In [ ]:
generate_summary = []
for i in original_text:
  text = pipe(i, truncation=True, min_length=100, do_sample=False)
  generate_summary.append(text)

In [ ]:
generate_summary = [i[0]['summary_text'] for i in generate_summary]

In [ ]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

In [ ]:
from rouge_score import rouge_scorer

def Calculate_rouge(reference_list, generate_list):
  scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
  rouge1 = []
  rouge2 = []
  rougeL = []
  n = len(reference_list)
  for i in range(n):
    scores = scorer.score(reference_list[i], generate_list[i])
    rouge1.append(scores['rouge1'].fmeasure)
    rouge2.append(scores['rouge2'].fmeasure)
    rougeL.append(scores['rougeL'].fmeasure)
  return rouge1, rouge2, rougeL

In [ ]:
rouge1, rouge2, rougeL = Calculate_rouge(gold_summary, generate_summary)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = TfidfVectorizer()
cos_sim_gold_w_generated = []

for i in range(len(generate_summary)):
  tfidf_matrix = vectorizer.fit_transform([gold_summary[i], generate_summary[i]])
  cos_sim_gold_w_generated.append(float(cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])))

In [ ]:
specific_example = pd.DataFrame()


In [ ]:
gold_summary[1]

'FINRA found the firm failed to reasonably supervise for potentially manipulative trading from 2013 through 2019 and SageTrader also failed to establish and implement AML policies and procedures reasonably expected to detect and cause the reporting of suspicious activity and violated FINRA Rules 3310(a) and 2010.  As a result, SageTrader consented to a censure and a $100,000 fine to FINRA.  Additionally, SageTrader also had to pay a total fine of $775,000 which was paid to FINRA and eight exchanges, and the firm was also required to review and revise its supervisory system for detecting potentially manipulative trading by the firm’s customers.'

In [ ]:
generate_summary[1]

'SageTrader has been a FINRA member since May 2006. FINRA found that the firm failed to reasonably supervise for potentially manipulative trading from 2013 through 2019. The AWC imposed a censure and a fine of $775,000 (paid to FINRA and eight exchanges) The firm considered two of the four customers to be high AML risk. The firm’s market participant identifiers (MPIDs) were used to route orders to anAlternative trading system and to exchanges. SageTrader failed to establish and implement AML policies and procedures reasonably expected to detect and cause the reporting of suspicious activity.'

In [ ]:
sum(cos_sim_gold_w_generated)/len(cos_sim_gold_w_generated)

0.5266007054740235

In [ ]:
from nltk.translate.meteor_score import meteor_score
from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import word_tokenize
import nltk
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
meteor = []
for i in range(len(generate_summary)):
  score = meteor_score([gold_summary[i].split()], generate_summary[i].split())
  meteor.append(score)

In [ ]:
meteor

[0.3493281731610533,
 0.4450648055832503,
 0.39856057004110457,
 0.2578635620915033,
 0.2543464320280727,
 0.20516359658050068,
 0.3157814867390296,
 0.3812592714406569,
 0.16092193621847461,
 0.16463025696108405,
 0.15881358763648773,
 0.20895548301965416,
 0.11212128961860438,
 0.1540524675487437,
 0.16439930234944086,
 0.2810754406814051,
 0.30987370783718066,
 0.272832904428312,
 0.3247585854706955,
 0.34358581440545094,
 0.568615163816691,
 0.41005298355032815,
 0.2348011191083576,
 0.6109129288571797,
 0.13043478260869568,
 0.6360027898008164,
 0.21020621676618245,
 0.3653001532977833,
 0.20729205220400132,
 0.09912967723014818,
 0.49443684179211045]

In [ ]:
bleu = []
for i in range(len(generate_summary)):
  score = sentence_bleu([word_tokenize(gold_summary[i])], word_tokenize(generate_summary[i]))
  bleu.append(score)

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [ ]:
bleu

[0.25396289656975746,
 0.40322629432644547,
 0.293473116017283,
 0.1274501455049666,
 0.20948078273304166,
 0.1305704919208898,
 0.02560856307185631,
 0.18087243451680823,
 0.0868748751703877,
 2.8309430953212897e-155,
 0.041794496634939816,
 9.68168896094018e-79,
 9.322729938094004e-79,
 0.020973193523444313,
 0.04412889691277824,
 0.1584970236319728,
 0.11450291363982446,
 0.14361850403564116,
 0.25643015581543144,
 0.199020627551958,
 0.3707109739058474,
 0.23723572432556672,
 0.13843322536976244,
 0.3895508854586635,
 4.675078406747889e-155,
 0.4053163208340827,
 0.08327986213702221,
 0.2217528915108828,
 0.06811144194104864,
 0.013580587471276367,
 0.4031283218746713]

In [ ]:
result = pd.DataFrame()
result['Rouge1 F1 score'] = rouge1
result['Rouge2 F1 score'] = rouge2
result['RougeL F1 score'] = rougeL
result['Cosine Similarity'] = cos_sim_gold_w_generated
result['Meteor'] = meteor
result['BLEU'] = bleu

In [ ]:
result.mean(4)

Rouge1 F1 score      0.461183
Rouge2 F1 score      0.293901
RougeL F1 score      0.340293
Cosine Similarity    0.526601
Meteor               0.297760
BLEU                 0.161987
dtype: float64

In [ ]:
cos_sim_gold_w_source

[array([[0.53736795]]),
 array([[0.56529796]]),
 array([[0.64003591]]),
 array([[0.59402518]]),
 array([[0.57431955]]),
 array([[0.57408963]]),
 array([[0.64375807]]),
 array([[0.70387225]]),
 array([[0.60091216]]),
 array([[0.61601896]]),
 array([[0.58018698]]),
 array([[0.4976579]])]

In [ ]:
cos_sim_generated_w_source

[array([[0.51266349]]),
 array([[0.64287477]]),
 array([[0.57801287]]),
 array([[0.55056563]]),
 array([[0.5853182]]),
 array([[0.6020538]]),
 array([[0.63479347]]),
 array([[0.60428287]]),
 array([[0.67346045]]),
 array([[0.72989717]]),
 array([[0.56958879]]),
 array([[0.51952124]])]

In [ ]:
generate_summary[0][0]['summary_text']

'Rosenblatt Securities Inc. has been registered with FINRA since 1987. The firm is headquartered in New York, New York and employs about 60registered representatives. Rosenblatt violated FINRA Rules 5123, 3110, and 2010 by failing to timely file offering documents for 12 private offerings as required by FINRA Rule 5123. By failing to establish and maintain a reasonable supervisory system and failing to file a private placement memorandum or any other document for the 12 offerings, RosenBlatt violated the rules.'

In [ ]:
!pip install seaplane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.8/92.8 kB 10.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 81.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.5/276.5 kB 